In this exercise, suppose you are a data scientist working for a city's government. They are interested in boosting levels of cycling, and want to make sure enough bikes are available for the public at any given time. You have been tasked with building a neural network model to predict the total number of bike rentals in the city, based on historical data. We have provided you with data recorded at the hour-level, which includes variables such as the weather, temperature, and time of day.

Your model should be designed to predict the total number of city bike rentals, on an hourly basis (i.e. each observation in the test/train data is the total number of bikes being rented for a given hour). You should therefore focus on predicting the variable 'cnt' -- which is the total count of rentals.

Further information on this dataset, including variable descriptions, can be found on the UCI Machine Learning Repository here (https://archive.ics.uci.edu/dataset/275/bike+sharing+dataset). You should read this description carefully, and be mindful of your choice of variables etc.

We have separated out a random test set from this data. Your goal is to train as predictive a model as possible using the provided formative_train.csv file. Once you are happy with your model, you should then predict the values for the formative_test.csv file. Note this file does not contain values for the outcome of interest.

Specifically, you should:

Define a neural network model class called my475_mod that is capable of taking in data, passing it through a neural network, and outputting a prediction.
You may rely on functions outside this class definition to help you build/train the model
Train an instance of that model on the provided training data and save your final trained model checkpoint to a file (this file should be named "<CANDIDATE_NUMER>.pt")
Pass our test data--formative_test.csv--through your trained model and save the predictions as a .csv file. The file should contain a single column of predictions, with each row corresponding to a row in the test data. The first row should contain the header "prediction". Save this file as "<CANDIDATE_NUMER>.csv"
We will assess both the quality of your code (i.e. the legibility of your definitions, the soundness of the architecture etc.) and the performance of your model on the test set.

In [4]:
import pandas as pd 
# Replace the path with the actual path to your formative_train.csv file
train_data = pd.read_csv('/Users/alyssapadbidri/Local/bike-rental-prediction/data/formative_train.csv')

# Display the first few rows to ensure it's loaded correctly
print(train_data.head())

       dteday  season  yr  mnth  hr  holiday  weekday  workingday  weathersit  \
0  2012-06-06       2   1     6   4        0        3           1           1   
1  2012-03-25       2   1     3   3        0        0           0           3   
2  2011-02-14       1   0     2  15        0        1           1           1   
3  2011-12-22       1   0    12   2        0        4           1           1   
4  2011-12-10       4   0    12   0        0        6           0           2   

   temp   atemp   hum  windspeed  casual  registered  cnt  
0  0.46  0.4545  0.82     0.1045       3           7   10  
1  0.42  0.4242  0.94     0.2985       8          10   18  
2  0.56  0.5303  0.21     0.6567      19          71   90  
3  0.48  0.4697  0.82     0.1045       1          10   11  
4  0.26  0.2727  0.81     0.1045      11          66   77  


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the data
train_data = pd.read_csv('/Users/alyssapadbidri/Local/bike-rental-prediction/data/formative_train.csv')
test_data = pd.read_csv('/Users/alyssapadbidri/Local/bike-rental-prediction/data/formative_test.csv')

# Remove 'dteday' column from both train and test data
train_data = train_data.drop('dteday', axis=1)
test_data = test_data.drop('dteday', axis=1)

# Extract features and target
X = train_data.drop('cnt', axis=1)  # All columns except 'cnt' are features
y = train_data['cnt']  # Target variable is 'cnt'

# Standardize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model
class My475Model(nn.Module):
    def __init__(self, input_dim):
        super(My475Model, self).__init__()
        # Define layers of the neural network
        self.layer1 = nn.Linear(input_dim, 64)  # Input layer to hidden layer (64 neurons)
        self.layer2 = nn.Linear(64, 32)         # Hidden layer (32 neurons)
        self.output_layer = nn.Linear(32, 1)    # Output layer with 1 output (cnt)

    def forward(self, x):
        # Define the forward pass through the layers with ReLU activation
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.output_layer(x)
        return x

# Initialize the model
input_dim = X_train.shape[1]  # Number of features
model = My475Model(input_dim)

In [7]:
# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

# Set up the loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100  # Number of epochs to train
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    optimizer.zero_grad()  # Zero the gradients

    # Forward pass
    y_pred = model(X_train_tensor)

    # Calculate the loss
    loss = criterion(y_pred, y_train_tensor)

    # Backward pass
    loss.backward()
    optimizer.step()

    # Print the loss every 10 epochs
    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

Epoch 1/100, Loss: 68449.890625
Epoch 11/100, Loss: 68322.75
Epoch 21/100, Loss: 68139.3515625
Epoch 31/100, Loss: 67860.0078125
Epoch 41/100, Loss: 67435.6875
Epoch 51/100, Loss: 66807.359375
Epoch 61/100, Loss: 65900.7265625
Epoch 71/100, Loss: 64628.05859375
Epoch 81/100, Loss: 62894.5546875
Epoch 91/100, Loss: 60605.80078125


In [8]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    y_val_pred = model(X_val_tensor)
    val_loss = criterion(y_val_pred, y_val_tensor)

print(f'Validation Loss: {val_loss.item()}')

Validation Loss: 58181.08984375
